In [6]:
import json,requests
from datetime import timedelta, datetime
from operator import itemgetter
import pandas as pd
def createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters):
	data = {}
	data['sources'] = sources
	data['metrics'] = metrics
	i = 1
	dimensionsDict = {}
	for dimension in dimensions:
		dimensionsDict[dimension] = i
		i += 1
	data['dimensions'] = dimensionsDict
	date1 = datetime.strptime(startDate,'%Y-%m-%d').strftime("%Y-%m-%d")
	date2 = (datetime.strptime(endDate,'%Y-%m-%d') + timedelta(1)).strftime("%Y-%m-%d")
	segments = []
	segmentsDict = {}
	segmentsDict['dateStart'] = "{0}T00:00:00+03:00".format(date1)
	segmentsDict['dateEnd'] = "{0}T00:00:00+03:00".format(date2)
	segmentsDict['filters'] = {}
	for filterKey, filterValue in filters.iteritems():
		segmentsDict['filters'][filterKey] = {}
		segmentsDict['filters'][filterKey]['type'] = 'DIMENSION'
		segmentsDict['filters'][filterKey]['ids'] = filterValue
	segments.append(segmentsDict)
	data['segments'] = segments
	return data

def getJSONOlapResponse(data):
	url = 'http://192.168.10.6:4000/api/OLAP/Plain'
	headers = {'Content-Type': 'application/json'}
	response = requests.post(url,data,headers=headers)
	return json.loads(response.text)

def transformOlapResponse(response,payload):
	data = []
	dimensionsList = [{'key': k, 'value': v} for k,v in payload['dimensions'].iteritems()]
	dimensionsList = sorted(dimensionsList, key=itemgetter('value'))
	for item in response:
		i = 0
		dataDict = {}
		for dimensionValue in item['dimensions']:
			dimensionName = dimensionsList[i]['key']
			dataDict[dimensionName] = dimensionValue
			i += 1
		metrics = item['value'][0]
		for k,v in metrics.iteritems():
			dataDict[k] = v
		data.append(dataDict)
	return data

In [65]:
filters = {'campaignTypeId': ['1','2'], 'campaignExternalId': ['16835371', '16082449', '15393268', '16040988', '16113633', '16040937', '16029140', '15220880', '14577051', '15220868', '15868173', '16838054', '15249945', '15808246', '17488134', '17384892', '16222583', '14268267', '16890155',
 '16117599', '15971087', '15335119', '15065611', '17295306', '14495748', '15868067', '15035707', '16040962', '16705155', '14022527', '17486641', '14827987', '17562154', '14643387', '17527977', '12525318', '11626791', '13229901',
 '15410338', '17395243', '17173805', '16729953', '16751863', '16838064', '17010970', '17580673', '17124644', '17358980', '17487662', '15101922', '16516395', '17580645', '15542270', '15220866', '16866188', '17563338', '12063095',
 '16222581', '17524965', '15220989', '15376198', '14589608', '15335189', '16651038', '16397636', '17082870', '14207460', '14940966', '16397643', '16455287', '15046894', '16688211', '16000698', '15228083', '15492867', '12060513',
 '16041000', '17437933', '15373660', '11502538', '16235646', '16041012', '16634519', '16001043', '16352459']}
sources = ["direct_conversion","metrika"]
metrics = ["clicks","spent","shows","visits","cpaVisits"]
dimensions = ["campaignExternalId","campaignName","keyPhraseName","searchPhraseName"]
startDate = '2016-02-14'
endDate = '2016-04-13'
payload = createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters)
response = getJSONOlapResponse(json.dumps(payload))
data = transformOlapResponse(response,payload)
data_df = pd.DataFrame(data)

In [66]:
data_df[data_df['visits'] > 0].dropna().to_excel('D:\\nch-research3.xlsx')

In [57]:
data_df['campaignExternalId'].unique()

array([u'14996336', u'15065611', u'15393268', u'16029140', u'16040937',
       u'16040988', u'16113633', u'17295306', u'17358980', u'17384892',
       u'17395243', u'17487662', u'17580673'], dtype=object)

In [61]:
filters = {'campaignTypeId': ['1','2']}
sources = ["direct_conversion","metrika"]
metrics = ["clicks","spent","shows","visits","cpaVisits"]
dimensions = ["campaignExternalId"]
startDate = '2016-02-14'
endDate = '2016-04-13'
payload = createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters)
response = getJSONOlapResponse(json.dumps(payload))
data = transformOlapResponse(response,payload)
df = pd.DataFrame(data)

In [64]:
list(df.sort_values(['cpaVisits'],ascending=False)['campaignExternalId'][15:100].as_matrix())

[u'16835371',
 u'16082449',
 u'15393268',
 u'16040988',
 u'16113633',
 u'16040937',
 u'16029140',
 u'15220880',
 u'14577051',
 u'15220868',
 u'15868173',
 u'16838054',
 u'15249945',
 u'15808246',
 u'17488134',
 u'17384892',
 u'16222583',
 u'14268267',
 u'16890155',
 u'16117599',
 u'15971087',
 u'15335119',
 u'15065611',
 u'17295306',
 u'14495748',
 u'15868067',
 u'15035707',
 u'16040962',
 u'16705155',
 u'14022527',
 u'17486641',
 u'14827987',
 u'17562154',
 u'14643387',
 u'17527977',
 u'12525318',
 u'11626791',
 u'13229901',
 u'15410338',
 u'17395243',
 u'17173805',
 u'16729953',
 u'16751863',
 u'16838064',
 u'17010970',
 u'17580673',
 u'17124644',
 u'17358980',
 u'17487662',
 u'15101922',
 u'16516395',
 u'17580645',
 u'15542270',
 u'15220866',
 u'16866188',
 u'17563338',
 u'12063095',
 u'16222581',
 u'17524965',
 u'15220989',
 u'15376198',
 u'14589608',
 u'15335189',
 u'16651038',
 u'16397636',
 u'17082870',
 u'14207460',
 u'14940966',
 u'16397643',
 u'16455287',
 u'15046894',
 u'166

In [62]:
df.sort_values(['cpaVisits'],ascending=False)

,campaignExternalId,clicks,cpaVisits,goalVisits,shows,spent,visits
111,13034433,3401,716,716,1292555,27358.299995,2790
402,14916983,2822,471,471,10870,9248.209946,2454
273,14258995,8865,399,399,6460402,64416.020697,6073
514,15220986,14132,278,278,97447,30641.279968,11410
587,15311241,2263,195,195,896763,21234.320099,2123
388,14812706,8267,143,143,1164150,81633.239796,7340
109,13028870,471,86,86,5572,6825.210036,371
710,15808840,569,68,68,33079,9895.529980,424
439,14997721,7809,63,63,2223300,36622.900208,6547
893,16368801,8798,52,52,1931323,31561.980088,7817


In [42]:
df2 = pd.read_csv('D:\\adgroups_direct.csv')

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2871: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=

In [43]:
df2

,_id,status,minusWords.0,minusWords.1,minusWords.2,minusWords.3,minusWords.4,minusWords.5,minusWords.6,minusWords.7,...,minusWords.205,minusWords.206,minusWords.207,minusWords.208,name,campaignId,brokerType,regionIds.0,externalId,id
0,56c0b4142e978708f99899e7,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,сопровождение банкротства ооо,16504224,0,1,1206724576,e27ba9d5-ca84-4feb-9c16-3e5ce75f6d61
1,56c0b4142e978708f99899e8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,юридическая консультация +по банкротству,16504224,0,1,1206724573,0c39fba2-32b1-4922-ba7c-f14ddbec4e95
2,56c0b4142e978708f99899ea,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,правовое регулирование банкротства юридическог...,16504224,0,1,1206724566,2b1ac0da-8c9d-4ed0-87e1-000c8b62a8a9
3,56c0b4142e978708f99899eb,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,банкротство должника юридического лица,16504224,0,1,1206724562,d78e8b68-abdf-4c6f-9847-d77a2fcc89fc
4,56c0b4142e978708f99899ec,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,банкротство юридических лиц юридические компании,16504224,0,1,1206724558,6e654ad6-eab2-4d59-b6d1-6f220e049487
5,56c0b4142e978708f99899ed,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ведение дел +по банкротству,16504224,0,1,1206724554,d79aa7f1-c716-4496-bd18-73747c3a7f5e
6,56c0b4142e978708f99899ee,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,основные процедуры банкротства юридического лица,16504224,0,1,1206724549,16e836b7-5a01-40a3-b013-4ae398471b07
7,56c0b4142e978708f99899ef,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,банкротство юридического лица стоимость,16504224,0,1,1206724547,ce280b58-7125-41f1-8add-e48a9844e968
8,56c0b4142e978708f99899f0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,исковое заявление +о банкротстве юридического ...,16504224,0,1,1206724544,2f0bdabf-dbdd-4dfe-94e9-2ecca5fce119
9,56c0b4142e978708f99899f1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,банкротство юридических лиц адвокат,16504224,0,1,1206724540,f3e8b1ce-9d31-454f-80a7-d4236b65ad8c


In [50]:
import json
with open('D:\\adgroups_direct.json','r+') as data_file:
    data = json.load(data_file)

ValueError: Extra data: line 2 column 1 - line 30001 column 1 (char 294 - 8528592)